In [1]:
# %matplotlib widget
%matplotlib qt
import dataloader
import pandas as pd
import os
import numpy as np
sid = '08db4255286f'

In [2]:
acc_data = dataloader.acc_data_for_child(sid)
events = pd.read_csv('data/train_events.csv')
dataloader.annotate_sid(acc_data, events, sid)

In [4]:
acc_data.sample(5)

,series_id,step,timestamp,anglez,enmo,activity
13223,08db4255286f,13223,2018-11-06 08:21:55+00:00,22.843800,0.0085,Sleeping
180018,08db4255286f,180018,2018-11-16 00:01:30+00:00,-44.847698,0.0026,Waking
416490,08db4255286f,416490,2018-11-29 16:27:30+00:00,-57.917000,0.0069,Waking
397633,08db4255286f,397633,2018-11-28 14:16:05+00:00,-81.621696,0.0001,Sleeping
394835,08db4255286f,394835,2018-11-28 10:22:55+00:00,-47.163601,0.0000,Sleeping


In [3]:
def filter_data(acc_data:pd.DataFrame, tss: list[pd.Timestamp], window: pd.Timedelta):
    net = sum([(acc_data['timestamp'] >= ts-window) & (acc_data['timestamp'] < ts+window) for ts in tss])
    net = net==True
    return acc_data.loc[net]

In [4]:
filt_acc = filter_data(
    acc_data=acc_data,
    tss=pd.to_datetime(events.loc[events['series_id']==sid, 'timestamp'], utc=True),
    window=pd.Timedelta(1, 'hour')
)

In [20]:
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from tqdm.notebook import tqdm
from visual import Visualizaer1
fig, ax = plt.subplots()

artists = Visualizaer1.init_frame(ax, filt_acc)
print(ax.get_yticklabels())
ani = anim.FuncAnimation(
    func=Visualizaer1.draw_frame,
    fig=fig,
    frames=tqdm(filt_acc['timestamp'][::30]),
    fargs=(filt_acc, pd.Timedelta(20, 'min'), acc_data['anglez'].std(), 2*acc_data['enmo'].std(), *artists),
    blit=True,
    interval=100

)
plt.show()



[Text(0, -3.0, '−3.0'), Text(0, -2.5, '−2.5'), Text(0, -2.0, '−2.0'), Text(0, -1.5, '−1.5'), Text(0, -1.0, '−1.0'), Text(0, -0.5, '−0.5'), Text(0, 0.0, '0.0'), Text(0, 0.5, '0.5'), Text(0, 1.0, '1.0')]


  0%|          | 0/2400 [00:00<?, ?it/s]

In [37]:
import importlib
import visual
importlib.reload(visual)

<module 'visual' from 'd:\\Projects\\ChildMindSleep\\visual.py'>

In [6]:
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from tqdm.notebook import tqdm
from visual import Visualizer2

fig, ax = plt.subplots()
time_window = pd.Timedelta(30, 'min')
artists = Visualizer2.init_frame(ax, time_window)
frames = tqdm(filt_acc['timestamp'][::7])
ani = anim.FuncAnimation(
    fig=fig,
    func=Visualizer2.draw_frame,
    frames=frames,
    fargs=(filt_acc, 0.3, time_window, artists),
    blit=True,
    interval=10
)
ani.save('vids/filtered_activity.mp4')


  0%|          | 0/10286 [00:00<?, ?it/s]

In [16]:
events.loc[events['series_id']==sid]

,series_id,night,event,step,timestamp
398,08db4255286f,1,onset,11988.0,2018-11-06T02:39:00-0400
399,08db4255286f,1,wakeup,14388.0,2018-11-06T05:59:00-0400
400,08db4255286f,2,onset,28680.0,2018-11-07T01:50:00-0400
401,08db4255286f,2,wakeup,31320.0,2018-11-07T05:30:00-0400
402,08db4255286f,3,onset,44184.0,2018-11-07T23:22:00-0400
403,08db4255286f,3,wakeup,49128.0,2018-11-08T06:14:00-0400
404,08db4255286f,4,onset,60336.0,2018-11-08T21:48:00-0400
405,08db4255286f,4,wakeup,66324.0,2018-11-09T06:07:00-0400
406,08db4255286f,5,onset,79152.0,2018-11-09T23:56:00-0400
407,08db4255286f,5,wakeup,85272.0,2018-11-10T08:26:00-0400


In [7]:
events.loc[events['series_id']==sid].__len__()

50